 # Assignment 6
    Group member: Akhila Ravi, Ruijia Xiong, Gregory Howard

In [1]:
import pandas as pd
import numpy as np

## 1. EDA

In [2]:
df19 = pd.read_csv('brfss19.csv')
df21 = pd.read_csv('brfss21.csv')
print(df19.shape)
print(df21.shape)

(418268, 250)
(438693, 250)


In [5]:
# Compare the different column names in the two data frame
(set(df19.columns) | set(df21.columns)) - (set(df19.columns) & set(df21.columns))

{'CHCCOPD2', 'CHCCOPD3'}

    • The different of the column names existed in the two data frame are 'CHCCOPD2', 'CHCCOPD3'

    Since the two names stand for the same thing, we just need to replace the column name df['CHCCOPD3'] in df21 to df['CHCCOPD2]

In [6]:
df21.rename(columns = {'CHCCOPD3':'CHCCOPD2'}, inplace = True)

In [7]:
# Now the two df has the same column names, we can concat the two df.
df = pd.concat([df19, df21], ignore_index=True, sort=False)
df.head()

,Unnamed: 0,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,...,_VEGESU1,_FRTLT1A,_VEGLT1A,_FRT16A,_VEG23A,_FRUITE1,_VEGETE1,_FLSHOT7,_PNEUMO3,_AIDTST4
0,0,1.0,1.0,1182019,1,18,2019,1100.0,2019000001,2.019000e+09,...,114.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0
1,1,1.0,1.0,1132019,1,13,2019,1100.0,2019000002,2.019000e+09,...,121.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0
2,2,1.0,1.0,1182019,1,18,2019,1100.0,2019000003,2.019000e+09,...,164.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,2.0
3,3,1.0,1.0,1182019,1,18,2019,1200.0,2019000004,2.019000e+09,...,NaN,9.0,9.0,1.0,1.0,1.0,1.0,9.0,9.0,NaN
4,4,1.0,1.0,1042019,1,4,2019,1100.0,2019000005,2.019000e+09,...,178.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0


### 1.1 Data processing with the chronic related columns
    - Depression: df['ADDEPEV3']
    - Ever Told Asthma: df['_CASTHM1']
    - Copd: df['CHCCOPD2']
    - Cancer: df['CHCSCNCR'], df['CHCOCNCR']
    - Ever Told Heart Condition: df['CVDCRHD4'], df['CVDINFR4'], df['CVDSTRK3']
    - Diabetes: df['DIABETE4']

In [12]:
# For cancer, we need to combine df['CHCSCNCR'] and f['CHCOCNCR']
print(df['CHCSCNCR'].unique())
print(df['CHCOCNCR'].unique())

[ 2.  1. nan  7.  9.]
[ 2.  1.  9.  7. nan]


    • According to the Codebook Report, only 1 stands for diagnosed. So the strategy to combine the columns would be either column is then the combined column is 1, otherwise, we use 0 to stand for not diagnosed.

In [27]:
# Combine for cancer
has_cncr = (df['CHCSCNCR'] == 1) | (df['CHCOCNCR'] == 1)
df['CNCR'] = has_cncr.astype(int)
df['CNCR'].value_counts()

0    708342
1    148619
Name: CNCR, dtype: int64

In [28]:
# Combine for heart condition
has_cvd = (df["CVDCRHD4"] == 1) | (df["CVDINFR4"] == 1) | (df["CVDSTRK3"] == 1) 
df["CVD"] = has_cvd.astype(int)
df["CVD"].value_counts() 

0    761194
1     95767
Name: CVD, dtype: int64

In [30]:
# print the percentage of either value in the two columns for cancer and heart condition
print(df['CNCR'].value_counts() / len(df['CNCR']))
print(df['CVD'].value_counts() / len(df['CVD']))

0    0.826574
1    0.173426
Name: CNCR, dtype: float64
0    0.888248
1    0.111752
Name: CVD, dtype: float64


• Before creating the two additional categorical columns, we need to clean the Nan values in the chronic condition columns

#### Missing value analysis in the chronic conditions

In [32]:
chronic_df = df[['ADDEPEV3', '_CASTHM1', 'CHCCOPD2', 'CHCSCNCR', 'CHCOCNCR', 'CNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3', 'CVD', 'DIABETE4']]
print(chronic_df.shape)

(856961, 11)


In [36]:
# Missing value analysis in the chronic_df
# list all the columns and number of missing values in that column
missing_cols = chronic_df.isna().sum().reset_index()
missing_cols

,index,0
0,ADDEPEV3,13
1,_CASTHM1,0
2,CHCCOPD2,11
3,CHCSCNCR,10
4,CHCOCNCR,12
5,CNCR,0
6,CVDCRHD4,10
7,CVDINFR4,12
8,CVDSTRK3,13
9,CVD,0


• We can see that there are only few missing values(<14) in these columns. We should just drop these rows.

In [37]:
df.dropna(subset=['ADDEPEV3', '_CASTHM1', 'CHCCOPD2', 'CHCSCNCR', 'CHCOCNCR', 'CVDCRHD4', 'CVDINFR4', 'CVDSTRK3', 'DIABETE4'], inplace=True)

In [39]:
### 1.2 Create two additional categorical columns - CMB_bianry, CMN_multi
has_chronic = (df['CNCR'] == 1)  | (df['ADDEPEV3'] == 1) | (df['_CASTHM1'] == 1) | (df['CHCCOPD2'] == 1) | (df['CVD'] == 1) | (df['DIABETE4'] == 1)
df['CMB_binary'] = has_chronic.astype(int)
df['CMB_binary'].unique()

array([1, 0])